In [ ]:
import xarray as xr
import zarr
import os
from utils import pyramid
import itertools

In [ ]:
root_path = "s3://carbonplan-benchmarks/data"

In [ ]:
attrs = {
    "variable": "tasmax",
    "model": "ACCESS-CM2",
    "data_type": "pyramids",
    "variant": "r1i1p1f1",
    "zarr_version": "v2",
    "shard_orientation": "0",
    "data_shard_size": "0",
    "data_dtype": "f4",
    "coordinate_chunks": "0",
    "coordinate_shards": "0",
    "compression": "gzipL1",
    "inflevel": "100",
}
pixels_per_tile = [128, 256]
data_chunk_size = [1, 5, 10, 25]
projections = ["3857", "4326"]
write_empty_chunks = [True]

In [ ]:
store = f"NEX-GDDP-CMIP6/{attrs['model']}/historical/{attrs['variant']}/{attrs['variable']}/{attrs['variable']}_day_{attrs['model']}_historical_r1i1p1f1_gn"
fp = os.path.join(root_path, store, "data-1a")
print(fp)

In [ ]:
for pix, chunk, proj, write in itertools.product(
    pixels_per_tile, data_chunk_size, projections, write_empty_chunks
):
    path = f"{attrs['data_type']}-{attrs['zarr_version']}-{proj}-{write}-{pix}-{chunk}-{attrs['shard_orientation']}-{attrs['data_shard_size']}-{attrs['data_dtype']}-{attrs['coordinate_chunks']}-{attrs['coordinate_shards']}-{attrs['compression']}-{attrs['inflevel']}"
    target = os.path.join(root_path, store, path)
    if proj == "4326":
        projection = 'equidistant-cylindrical'
    elif proj == "3857":
        projection = 'web-mercator'
    pyramid(
        ds_path=fp,
        target=target,
        levels=4,
        pixels_per_tile=pix,
        target_mb=chunk,
        projection=projection,
        extra_attrs=attrs,
    )